In [7]:
# quick and dirty way to change the current working directory to root (/toxic-comment-classification)
# you should run this at least once just to be certain
from os import chdir, path, getcwd
if getcwd().endswith("src"):
    chdir(path.pardir)
if path.isfile("checkcwd"):
    print("Success")
else:
    raise Exception("Something went wrong. cwd=" + getcwd())

Success


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from gc import collect

In [9]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=10000,
    )

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(2, 6),
    max_features=50000,
    )

In [10]:
path = 'kaggle/input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
clean_data_path = 'clean_data/'
EMBEDDING_FILE=f'{path}glove_embeddings/glove.6B.300d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv.zip'
TEST_DATA_FILE=f'{path}{comp}test.csv.zip'
CLEAN_TRAIN_DATA_FILE=f'{clean_data_path}data_train_cleaned_vanilla2.txt'
CLEAN_TEST_DATA_FILE=f'{clean_data_path}data_test_cleaned_vanilla2.txt'
SAMPLE_SUBMISSION=f'{path}{comp}sample_submission.csv.zip'

In [11]:
def read_from_file(filename):
    with open(filename, 'r') as f:
        return pd.Series(f.read().splitlines())

In [12]:
X_train = read_from_file(CLEAN_TRAIN_DATA_FILE)
X_test = read_from_file(CLEAN_TEST_DATA_FILE)
full_text = np.concatenate([X_train, X_test])

word_vectorizer.fit(full_text)
char_vectorizer.fit(full_text)

collect()

train_word_features = word_vectorizer.transform(X_train)
train_char_features = char_vectorizer.transform(X_train)

test_word_features = word_vectorizer.transform(X_test)
test_char_features = char_vectorizer.transform(X_test)


X_t = sparse.hstack([train_word_features, train_char_features]).toarray()
X_te = sparse.hstack([test_word_features, test_char_features]).toaray()

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

: 

In [ ]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)
y = train[list_classes].values

In [ ]:
from sklearn.model_selection import KFold
num_folds = 10

collect()

scores = []
oof_predict = np.zeros((train.shape[0], 6))

predict = np.zeros((test.shape[0], 6))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_t):
    
    kfold_y_train, kfold_y_test = y[train_index], y[test_index]
    kfold_X_train = X_t[train_index]

    kfold_X_valid = X_t[test_index]

    collect()

    for i, col in enumerate(list_classes):
        model = LogisticRegression(C=2, random_state=42, class_weight = 'balanced')
        model.fit(kfold_X_train, kfold_y_train[col])
        predict[i] += model.predict_proba(X_te)

# not done yet